## Introduction to Optimization

### Goal

The goal of optimization is to find either the maximum or minimum $x^*$ of a given function $f(x)$. For a local optimum (minimum/maximum) we know from calculus, that the derivative of the function must be zero $f'(x^*)=0$. In simple cases this allows us to find the optimum analytically, e.g.
$$f(x) = x^2 \qquad \Longrightarrow \qquad f'(x)=2x \qquad \Longrightarrow \qquad x^*=0.$$
However, in many cases this might not be possible anymore such that an optimum can only be found through computational methods.

### Gradient Method
Given a function of several variables $f(x) \in \mathbb{R}$ with $x \in \mathbb{R}^n$, i.e. $f(x_1,x_2,...,x_n)$, the gradient $\nabla f(x)$ is understood as the derivative of the function. This also means that a local optimum $x^*$ satisfies $\nabla f(x^*)$. It is defined as follows
$$\nabla f(x) = \begin{pmatrix}
\partial_{x_1}f(x) \\
\vdots \\
\partial_{x_n}f(x)
\end{pmatrix}.$$
Here, $\partial_{x_i}f(x)$ denotes the partial derivative with respect to $x_i$.

#### Example
$$f(x) = \cos(x_1) + x_2^2 \qquad \Longrightarrow \qquad \partial_{x_1} f(x) = -\sin(x_1), \qquad \partial_{x_2} f(x) =2x_2.$$
In this example the gradient will be
$$\nabla f(x) = \begin{pmatrix}
 -\sin(x_1) \\
2x_2
\end{pmatrix}.$$


In 2D such a function can be visualized using a contourplot.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x1 = np.linspace(0,2*np.pi,100)
x2 = np.linspace(-2,2,100)

X1, X2 = np.meshgrid(x1, x2)

f = lambda x: np.cos(x[0]) + x[1]**2

plt.figure(figsize=(9, 7))
plt.contourf(X1, X2, f([X1, X2]), 50, cmap='jet', alpha = 0.3)
plt.colorbar()
plt.contour(X1, X2, f([X1, X2]), 50, colors='black', linewidths=1, alpha=0.3)

# Choose some point
point = np.array([5, 1.3])

# Calculate gradient at that point
gradient = np.array([-np.sin(point[0]), 2*point[1]])

# Plot the negative gradient
plt.quiver(*point, *-gradient, angles='xy', scale_units='xy', scale=1, color='k')

The gradient is a vector that always points in the direction of the largest slope. Therefore, following the negative gradient should lead to the minimum of a function. We can do this by starting with any point $x^{(0)}$ and update this point in every iteration such that
$$x^{(k+1)} = x^{(k)} - \alpha \nabla f(x^{(k)})$$

In [ ]:
x1 = np.linspace(0,2*np.pi,100)
x2 = np.linspace(-2,2,100)

X1, X2 = np.meshgrid(x1, x2)

plt.figure(figsize=(9, 7))
plt.contourf(X1, X2, f([X1, X2]), 50, cmap='jet', alpha = 0.3)
plt.colorbar()
plt.contour(X1, X2, f([X1, X2]), 50, colors='black', linewidths=1, alpha=0.3)

# Define gradient
df = lambda x: np.array([-np.sin(x[0]), 2*x[1]])

# Choose some starting point
x0 = np.array([2, -1.7])

# step size (in machine learning it is called learning rate)
α = 0.3

for k in range(10):
    plt.quiver(*x0, *-α*df(x0), angles='xy', scale_units='xy', scale=1, color='k')
    x0 = x0 - α*df(x0)
    
print('Function values =', f(x0))
print('Gradient = ', df(x0))

### Step size strategy (Armijo Rule)

Using a fixed step size $\alpha$ often leads to oscillations around the local minimum such that the true value will never be found. Adaptive step sizes allow us to avoid this problem and guarantee that the method converges towards the minimum. The Armijo rule goes like this
\begin{align*}
\text{if} \qquad &f(x - \alpha^{(j)} \nabla f(x))  - f(x) > -c \alpha^{(j)} \nabla f(x) \cdot \nabla f(x)\\
\text{then} \qquad &\alpha^{(j+1)} = 0.5\alpha^{(j)}.
\end{align*}
Parameter $c$ is usually chosen to be small, e.g. $10^{-4}$.

In [ ]:
x1 = np.linspace(0,2*np.pi,100)
x2 = np.linspace(-2,2,100)

X1, X2 = np.meshgrid(x1, x2)

plt.figure(figsize=(9, 7))
plt.contourf(X1, X2, f([X1, X2]), 50, cmap='jet', alpha = 0.3)
plt.colorbar()
plt.contour(X1, X2, f([X1, X2]), 50, colors='black', linewidths=1, alpha=0.3)

# Define gradient
df = lambda x: np.array([-np.sin(x[0]), 2*x[1]])

# Choose some starting point
x0 = np.array([np.random.uniform(0,2*np.pi), np.random.uniform(-2,2)])

for k in range(10):
    α = 1
    x = x0 - α*df(x0)
    while f(x) - f(x0) > -1e-4*α*np.dot(f(x0), f(x0)):
        α = α/2
        x = x0 - α*df(x0)
    plt.quiver(*x0, *-α*df(x0), angles='xy', scale_units='xy', scale=1, color='k')
    x0 = x#x0 - α*df(x0)
    
print('Function values =', f(x0))
print('Gradient = ', df(x0))
print('Solution = ', x0)

### Complete function (without plot)

In [ ]:
def gradient_method(f, df, x0, tol=1e-8, maxit=20):
    norm = 1
    norm = np.dot(df(x0), df(x0))**(1/2)
    k = 1
    while norm > tol and k < maxit+1:
        α = 1
        x = x0 - α*df(x0)
        while f(x) - f(x0) > -1e-4*α*norm**2:
            α = α/2
            x = x0 - α*df(x0)
            
        x0 = x
        norm = np.dot(df(x0), df(x0))**(1/2)
        print('Iteration = ' + str(k) + '  Norm = ' + str(norm))
        k += 1
            
    return x0
              
x = gradient_method(f, df, np.array([0.8,-1.7]))
print(x)

### Higher order methods
The gradient method is a first order optimization method, i.e. the error towards the optimum reduces linearly in every step. For faster convergence we want to make us of a higher order method.

#### Newton's method
Newton's method is a (locally) second order method that was originally not designed to minimize a function but instead solve equations of the form
$$F(x) = 0, \qquad \text{with } F(x) \in \mathbb{R^n} \text{ and } x \in \mathbb{R^n}.$$
The derivative of $F$ will therefore be a matrix with $n$ rows and $n$ columns, i.e. $F'(x) \in \mathbb{R^{n\times n}}$. It is usually referred to as the Jacobi matrix of $F$ and is defined like this
$$F'(x) :=
\begin{pmatrix}
\dfrac{\partial F_1}{\partial x_1}(x)&\cdots&\dfrac{\partial F_1}{\partial x_n}(x)\\
\vdots&\ddots &\vdots\\
\dfrac{\partial F_n}{\partial x_1}(x)&\cdots&\dfrac{\partial F_n}{\partial x_n}(x)
\end{pmatrix}$$
Similar to the gradient method, Newton's method follows a simple iteration with
$$x^{(k+1)} = x^{(k)} - F'(x^{(k)})^{-1}F(x^{(k)})$$


In [ ]:
from numpy.linalg import solve, norm

def Newton(F, dF, x0, tol=1e-4, maxit=20):
    k = 0
    res = 1
    x0 = np.array(x0)
    fnorm = norm(F(x0))
    print('Iteration =', k, 'Residual =', res, ', Norm =', fnorm)
    while k < maxit+1 and res > tol and fnorm > 1e-3*tol:
        if x0.shape == ():
            dx0 = -F(x0)/dF(x0)
            res = abs(dx0/x0)
        else:
            dx0 = -solve(dF(x0), F(x0))
            res = norm(dx0)/norm(x0)
            
        x = x0 + dx0
        x0 = x
        k += 1
        fnorm = norm(F(x0))
        print('Iteration =', k, 'Residual =', res, ', Norm =', fnorm)
        
    return x

F = lambda x: np.array([np.sin(x[0]), np.cos(x[1])])
dF = lambda x: np.array([[np.cos(x[0]), np.cos(x[1])],
                        [-np.sin(x[0]), -np.sin(x[1])]])

x = Newton(F, dF, np.array([3,1]), tol = 1e-10)
print(x)

#### Application to optimization
Newton's method cannot be directly applied to an optimization problem. However, we know that an optimum $x^*$ satisfies $\nabla f(x^*) = 0$. Therefore, we define $F(x) := \nabla f(x)$ and solve this equation instead. The derivative $F'(x)$ is usually called the Hessian matrix and is usually denoted by $H_f(x)$. It is basically the Jacobian matrix of the Gradient $\nabla f$ and can be written as
$$H_f(x) :=
\begin{pmatrix}
\dfrac{\partial^2 f}{\partial x_1 \partial x_1}(x)&\cdots&\dfrac{\partial^2 f}{\partial x_1 \partial x_n}(x)\\
\vdots&\ddots &\vdots\\
\dfrac{\partial^2 f}{\partial x_n \partial x_1}(x)&\cdots&\dfrac{\partial^2 f}{\partial x_n \partial x_n}(x).
\end{pmatrix}$$
If the gradient $\nabla f$ desrcibes first derivate of a function $f$, then the Hessian $H_f$ can be considered the second derivative.
##### Remark: The higher the order of a method, the more derivatives of a function we need!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

f = lambda x: np.cos(x[0]) + x[1]**2
df = lambda x: np.array([-np.sin(x[0]), 2*x[1]])
d2f = lambda x: np.array([[-np.cos(x[0]), 0],[0, 2]])

x0 =np.array([3,1])
x = Newton(df, d2f, x0, tol = 1e-10)
print(x)

### Final remarks
We have seen examples of optimization in two dimensions, with the code being generalized to $n$ dimensions. However, depending on the exact problem formulation, the concept of the gradient and second derivatives usually have to be broadened even further. Popular fields are
- constrained optimization
- optimization in Banach spaces
- optimization with (partial) differential equations
- inverse modeling